In [1]:
## ALL FUNCTIONS
import matplotlib.pyplot as plt

# List of unique colors for the lines
unique_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b',
                 '#e377c2', '#7f7f7f', '#bcbd22', '#17becf', '#aaffc3', '#ffaec9',
                 '#a9a9a9', '#ffd700', '#4682b4']

# Create a custom color cycle
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=unique_colors)

def queryServer(regions, new_data):
    """
    Return all data from the server in 6 dataframes (df_all, TAS, SA, QLD, NSW, VIC)
    
    If IP blocked: Azure> aemo-server sql > (security) > networking > (firewall rules) > add IP
    """
    # Connect to Server
    if new_data == False and os.path.exists("TAS_Data.csv") and os.path.exists("SA_Data.csv") and os.path.exists("QLD_Data.csv") and os.path.exists("NSW_Data.csv") and os.path.exists("VIC_Data.csv"):
        df_all =  pd.DataFrame()
        TAS =  pd.DataFrame()
        SA =  pd.DataFrame()
        QLD =  pd.DataFrame()
        NSW =  pd.DataFrame()
        VIC =  pd.DataFrame()
        return df_all, TAS, SA, QLD, NSW, VIC
    else:
        connection_string = ("ENTER YOUR CONNECTION STRING HERE")
        connection = pyodbc.connect(connection_string)

        # Call Data from Server
        query_all = 'SELECT * FROM AEMO_Price_Demand_Data'
        data_all = pd.read_sql(query_all, connection)
        data_all['SETTLEMENTDATE'] = pd.to_datetime(data_all['SETTLEMENTDATE']) # changes datetime format
        df_all = data_all.drop_duplicates(subset=['SETTLEMENTDATE', 'REGION'], keep='first') # deletes any duplicates

        # Seperate data frame by state
        TAS = df_all[df_all["REGION"] == 'TAS1']
        SA = df_all[df_all["REGION"] == 'SA1']
        QLD = df_all[df_all["REGION"] == 'QLD1']
        NSW = df_all[df_all["REGION"] == 'NSW1']
        VIC = df_all[df_all["REGION"] == 'VIC1']

        return df_all, TAS, SA, QLD, NSW, VIC


def timeFormatting(regions, new_data = False, check = True):
    """
    Resample df to make all data intervals 30 minutes
    Where data is missing (ie. intevals > 30 min), rows are created for the missing 30 min interval and demand = 0 and RRP = NaN (0)
    EXTREMELY SLOW TO format data, thus download and reupload csv, which will miss recent data
    In future could do it so that it only does recent data and runs every day etc.
    INPUTS:
        regions - list of regions
        new_data - Boolean check for if to append new data to existing data, if existing data exists
        check = True - checks that the function has worked properly by printing values of the output df where interval != 30
    OUTPUTS:
        global variables for each reach with data formatted to 30 minute intervals
    """

    dataframes_dict = {}
    
    # Select Region
    for df_name in regions:
        file_path = df_name + "_Data.csv"

        if os.path.exists(file_path):
            
            print(f'{df_name}: resampled data exists. Calling from csv.')
            df = pd.read_csv(file_path)
            df['SETTLEMENTDATE'] = pd.to_datetime(df['SETTLEMENTDATE'])
            
            # Get the last date in the existing CSV
            last_date_in_csv = df['SETTLEMENTDATE'].max()
            print(f"     CSV Last Date: {last_date_in_csv}")
          
            if new_data == True:
                # Get the data from temp_data after the last date in CSV
                temp_data = globals()[df_name]
                temp_data['SETTLEMENTDATE'] = pd.to_datetime(temp_data['SETTLEMENTDATE'])

                last_date = temp_data['SETTLEMENTDATE'].max()
                print(f"     SQL Last Date: {last_date}")

                if last_date != last_date_in_csv:
                    temp_data = temp_data[temp_data['SETTLEMENTDATE'] > last_date_in_csv]
                    temp_data.set_index('SETTLEMENTDATE', inplace=True)


                    # Define custom aggregation functions
                    sum_agg = lambda x: x.sum()  # For 'TOTALDEMAND'
                    weighted_avg_agg = lambda x: (np.sum(x * temp_data.loc[x.index, 'TOTALDEMAND'])) / np.sum(temp_data.loc[x.index, 'TOTALDEMAND'])  # For 'RRP'

                    # Resample 'TOTALDEMAND' using sum aggregation and 'RRP' using weighted average aggregation to 30 minute interval
                    resampled_df = temp_data.resample('30T', label='left', closed='left').agg({'TOTALDEMAND': sum_agg, 'RRP': weighted_avg_agg})
                    resampled_df.reset_index(drop=False, inplace=True)

                    # Remove last interval as to ensure consistent time periods
                    resampled_df = resampled_df.iloc[:-1]
                    resampled_df.to_csv(file_path, mode='a', header=False, index=False)
            
            globals()[df_name] = pd.read_csv(file_path)
            
            if check == True:
                # Check time intervals
                # Calculate the intervals in hours
                resampled_df = globals()[df_name]
                intervals = resampled_df['SETTLEMENTDATE'].diff() / pd.Timedelta(hours=1)

                # Check if any interval is not equal to 0.5 hours
                not_equal_to_half = intervals != 0.5

                # Print the rows where the interval is not equal to 0.5 hours
                print('Below sample not at 30 minute interval:')
                print(resampled_df[not_equal_to_half])
                print("")

        else:
            print(f'{df_name}: Resampling data to 30 min intervals, overwriting original data frames and saving csv files')
            # Change index to date
            temp_data = globals()[df_name]
            temp_data.set_index('SETTLEMENTDATE', inplace=True)

            # Define custom aggregation functions
            sum_agg = lambda x: x.sum()  # For 'TOTALDEMAND'
            weighted_avg_agg = lambda x: (np.sum(x * temp_data.loc[x.index, 'TOTALDEMAND'])) / np.sum(temp_data.loc[x.index, 'TOTALDEMAND'])  # For 'RRP'

            # Resample 'TOTALDEMAND' using sum aggregation and 'RRP' using weighted average aggregation to 30 minute interval
            resampled_df = temp_data.resample('30T', label='left', closed='left').agg({'TOTALDEMAND': sum_agg, 'RRP': weighted_avg_agg})
            resampled_df.reset_index(drop=False, inplace=True)

            # Remove first and last interval as to ensure consistent time periods
            resampled_df = resampled_df.iloc[1:-1]

            resampled_df.to_csv(file_path, index = False)
            print(f"Location: {df_name} saved as {output_file}")

            if check == True:
                # Check time intervals
                # Calculate the intervals in hours
                intervals = resampled_df['SETTLEMENTDATE'].diff() / pd.Timedelta(hours=1)

                # Check if any interval is not equal to 0.5 hours
                not_equal_to_half = intervals != 0.5

                # Print the rows where the interval is not equal to 0.5 hours
                print('Below sample not at 30 minute interval:')
                print(resampled_df[not_equal_to_half])
                print("")

             # Overwrite the original DataFrame with the modified DataFrame
            globals()[df_name] = resampled_df

    return TAS, SA, QLD, NSW, VIC


def combineRegions(regions):
    """
    Weighted average of dataset for entire NEM
    """
    data = pd.concat([VIC, NSW, SA, QLD, TAS], ignore_index=True)

    # Calculate total demand for each time step
    total_demand = data.groupby('SETTLEMENTDATE')['TOTALDEMAND'].sum()

    # Calculate the weighted average of prices using demand as weights
    weighted_price = (data['TOTALDEMAND'] * data['RRP']).groupby(data['SETTLEMENTDATE']).sum() / total_demand

    # Create a new DataFrame to store the combined values
    combined_data = pd.DataFrame({'TOTALDEMAND': total_demand, 'RRP': weighted_price})
    combined_data = combined_data.reset_index()
    combined_data.to_csv('AUS_Data.csv', index = False)

    return combined_data


def dayMonthRevenue(regions, rte, new_data):
    """
    Calculate daily revenue and average monthly revenue, and save as csv OR upload csv containing this data
    INPUTS:
        regions - list of regions
        rte - round trip efficiency of storage device being examined
        check = True - checks that the function has worked properly by printing values of the output df where interval != 30
    OUTPUTS:
        global variables for each reach with data formatted to 30 minute intervals
    
    """

    # Select Region
    for df_name in regions:
        file_path1 = df_name + "_" + str(rte) + "_DailyRevenue.csv"
        file_path2 = df_name + "_" + str(rte) + "_AvMonthlyRevenue.csv"

        if os.path.exists(file_path1) and os.path.exists(file_path2) and new_data == False:
            # Import Normalised Data
            dr_name = df_name + '_DR'
            aMR_name = df_name + '_aMR'

            globals()[dr_name] = pd.read_csv(file_path1)
            globals()[dr_name]['date'] = pd.to_datetime(globals()[dr_name]['date'])
            globals()[aMR_name] = pd.read_csv(file_path2)

        else:
            df = globals()[df_name]
            temp_data = df.copy()

            # Group the DataFrame by the 'date' column
            temp_data['SETTLEMENTDATE'] = pd.to_datetime(temp_data['SETTLEMENTDATE'])
            grouped_df = temp_data.groupby(temp_data['SETTLEMENTDATE'].dt.date)

            # Create a new DataFrame to store the results
            dailyRevenue_df = pd.DataFrame(columns=['date'] + [f'hours={n/2}' for n in range(1, 25)])

            # Iterate over each group
            for date, group in grouped_df:
                price_differences = {}  # Dictionary to store price differences for different values of 'n'
                for n in range(1, 25):
                    # Calculate the difference between the two highest and two lowest prices
                    price_difference = group.nlargest(n, 'RRP')['RRP'].sum() * rte - group.nsmallest(n, 'RRP')['RRP'].sum()

                    # Store the price difference in the dictionary
                    price_differences[f'hours={n/2}'] = price_difference*0.5 # as each interval is half an hour

                # Append the results to the result DataFrame
                dailyRevenue_df = dailyRevenue_df.append({'date': date, **price_differences}, ignore_index=True)

            # Convert the 'settlementdate' column to datetime if it's not already in datetime format
            dailyRevenue_df['date'] = pd.to_datetime(dailyRevenue_df['date'])

            # Create a new column for the month and Group the DataFrame by the month
            dailyRevenue_df['month'] = dailyRevenue_df['date'].dt.to_period('M')
            monthlyAvRevenue_df = dailyRevenue_df.groupby('month')

            # Calculate the average for each column
            avgRevMonth = monthlyAvRevenue_df.mean()
            dailyRevenue_df = dailyRevenue_df.drop('month', axis=1)

            dailyRevenue_df.to_csv(file_path1, index = False)
            avgRevMonth.to_csv(file_path2, index = True)
            print(f"Location: {df_name}")

            # Overwrite the original DataFrame with the modified DataFrame
            globals()[df_name + '_DR'] = dailyRevenue_df
            globals()[df_name + '_aMR'] = avgRevMonth
            
    return TAS_DR, SA_DR, QLD_DR, NSW_DR, VIC_DR, AUS_DR, TAS_aMR, SA_aMR, QLD_aMR, NSW_aMR, VIC_aMR, AUS_aMR       

def weighted_avg_and_std(values, weights):
    """
    Return the weighted average and standard deviation.

    values, weights -- NumPy ndarrays with the same shape.
    """
    average = numpy.average(values, weights=weights)
    # Fast and numerically precise:
    variance = numpy.average((values-average)**2, weights=weights)
    return (average, math.sqrt(variance))


def yearlyRevenuePlot(regions, financial_year, years):
    """
    Plot yearly total revenue according to number of hours of energy storage
    """
    
    # Extract the year from the settlementdate column
    for i, df_name in enumerate(regions):
        name = df_name + '_DR'
        df = globals()[name]
        dailyRevenue = df.copy()

        # Drop half hour increments of number of hours of storage
        drop_idx = list(range(1,dailyRevenue.shape[1],2)) #Indexes to drop
        drop_cols = [j for i,j in enumerate(dailyRevenue.columns) if i in drop_idx] #<--
        dailyRevenue_df = dailyRevenue.drop(drop_cols, axis=1)         #Drop them at once over axis=1
        
        # Rename columns by keeping only the numeric part
        dailyRevenue_df.columns = [col.split('=')[1] if 'hours=' in col else col for col in dailyRevenue_df.columns]
        
        if financial_year == True:
            # Define the start and end months of your financial year
            financial_year_start_month = 7  # July
            financial_year_end_month = 6    # June
            
            ## Calculate the financial year based on the date column
            dailyRevenue_df['year'] = (dailyRevenue_df['date'].dt.year - (dailyRevenue_df['date'].dt.month < financial_year_start_month))

            # Calculate the financial year based on the date column using the defined function
            #dailyRevenue_df['year'] = dailyRevenue_df.apply(calculate_financial_year, axis=1)
            x_label = 'Financial Year'


        else:
            dailyRevenue_df['year'] = dailyRevenue_df['date'].dt.year
            x_label = 'Calendar Year'

        # Group the DataFrame by year and n, and calculate the sum of revenue
        yearly_total_revenue = dailyRevenue_df.groupby('year').mean()

        # Calculate the marginal increase in revenue for each extra hour of storage
        
        # Drop incomplete year and unnecessary columns
        dailyRevenue_df = dailyRevenue_df.drop('year', axis=1)
        yearly_total_revenue.drop(yearly_total_revenue.tail(1).index,inplace=True)

        yearly_total_revenue.tail(years).plot.bar(figsize=(10, 6))
        y_label = 'Average Daily Revenue (A$/MW)' #? $AU / kW-year
 
        # Set plot labels and title
        plt_title = 'Value of Potential Revenue from Storage for ' + df_name
        #plt_title = 'Value of Potential Storage Revenue for the NEM'

        plt.xlabel(x_label)
        plt.ylabel(y_label)
        plt.title(plt_title)
        plt.legend(title='Hours of Storage', ncol=2)
        plt.grid(True)
        plt.show()  
        
        globals()[df_name + '_YtR'] = yearly_total_revenue
      # globals()[df_name + '_MIR'] = marginal_revenue_increase

        
def yearlyProfitPlot(regions, financial_year, years, lcos):
    """
    Plot yearly total revenue according to number of hours of energy storage
    """
    
    # Extract the year from the settlementdate column
    for i, df_name in enumerate(regions):
        name = df_name + '_DR'
        df = globals()[name]
        dailyRevenue = df.copy()
        for i, column in enumerate(dailyRevenue.columns[1:], start=2):
            dailyRevenue[column] = dailyRevenue[column] - (i-1) * (lcos/2)
        globals()[df_name + '_DP'] = dailyRevenue

        # Drop half hour increments of number of hours of storage
        drop_idx = list(range(1,dailyRevenue.shape[1],2)) #Indexes to drop
        drop_cols = [j for i,j in enumerate(dailyRevenue.columns) if i in drop_idx] #<--
        dailyRevenue_df = dailyRevenue.drop(drop_cols, axis=1)         #Drop them at once over axis=1
        
        # Rename columns by keeping only the numeric part
        dailyRevenue_df.columns = [col.split('=')[1] if 'hours=' in col else col for col in dailyRevenue_df.columns]
        
        if financial_year == True:
            # Define the start and end months of your financial year
            financial_year_start_month = 7  # July
            financial_year_end_month = 6    # June
            
            ## Calculate the financial year based on the date column
            dailyRevenue_df['year'] = (dailyRevenue_df['date'].dt.year - (dailyRevenue_df['date'].dt.month < financial_year_start_month))

            # Calculate the financial year based on the date column using the defined function
            #dailyRevenue_df['year'] = dailyRevenue_df.apply(calculate_financial_year, axis=1)
            x_label = 'Financial Year'


        else:
            dailyRevenue_df['year'] = dailyRevenue_df['date'].dt.year
            x_label = 'Calendar Year'

        # Group the DataFrame by year and n, and calculate the sum of revenue
        yearly_total_revenue = dailyRevenue_df.groupby('year').mean()

        # Calculate the marginal increase in revenue for each extra hour of storage
        
        # Drop incomplete year and unnecessary columns
        dailyRevenue_df = dailyRevenue_df.drop('year', axis=1)
        yearly_total_revenue.drop(yearly_total_revenue.tail(1).index,inplace=True)

        yearly_total_revenue.tail(years).plot.bar(figsize=(10, 6))
        y_label = 'Average Daily Profit (A$/MW)' #? $AU / kW-year
 
        # Set plot labels and title
        plt_title = 'Value of Potential Storage Profit for ' + df_name

        plt.xlabel(x_label)
        plt.ylabel(y_label)
        plt.title(plt_title)
        plt.legend(title='Hours of Storage', ncol=2)
        plt.grid(True)
        plt.show()  
        
        globals()[df_name + '_YtP'] = yearly_total_revenue
      # globals()[df_name + '_MIR'] = marginal_revenue_increase
    
    
def revenueDurationPlot(regions):
    """
    Plot duration of revenue for all states and all values of n
    https://blog.finxter.com/plotting-a-load-duration-curve-with-python/
    """

    # Sort the DataFrame by the loads, in descending order of magnitude
    years = range(2000, 2023)
    dict_of_df = {}  # initialize empty dictionary
    ns = range(2, 25, 2)

    for n in ns:
        storage_time = 'hours=' + str(n)+'.0'
        fig, axes = plt.subplots(nrows=1, ncols=len(regions), figsize=(15, 5))

        for i, df_name in enumerate(regions):
            df = globals()[df_name + '_DR']
            for year in years:
                DR_sorted = df.loc[df.date.dt.year == year]
                DR_sorted = DR_sorted.sort_values(by=['date'], ascending=True)
                DR_sorted = DR_sorted.sort_values(by=[storage_time], ascending=False)

                DR_sorted['duration'] = 1
                DR_sorted['duration'] = DR_sorted['duration'].cumsum()

                DR_sorted['CumRev'] = DR_sorted[storage_time].cumsum()
                DR_sorted['CumRevNorm'] = DR_sorted['CumRev'] / DR_sorted[storage_time].sum()*100

                dict_of_df["df_{}".format(year)] = DR_sorted

                # Plot the load duration curve (Revenue vs Percentage of time) on the corresponding subplot
                pltTitle = df_name
                 #Create the line plot
                lineplot = sb.lineplot(x="duration", y="CumRevNorm", data=dict_of_df['df_' + str(year)], ax=axes[i])

                # Set attributes using .set() on the AxesSubplot object
                lineplot.set(title=pltTitle, xlabel='Number of Days', ylabel='Percentage of yearly Revenue (%)')

                axes[i].set_ylim(0, 100)
                axes[i].set_xlim(0, 366)
                axes[i].set_title(pltTitle)
                axes[i].grid(True)  # Turn on gridlines


        plt.suptitle('Normalised Revenue Duration Curves for ' + str(int(n / 2)) + ' hours of storage')
        plt.tight_layout()
        plt.show()
        
def revenueDurationPlot2(region, n_values):
    """
    Plot duration of revenue for a single region and multiple values of n on the same graph
    """

    # Sort the DataFrame by the loads, in descending order of magnitude
    #years = [2000,2010,2020,2022]
    years = [2018, 2020, 2022]#,2022]

    dict_of_df = {}  # initialize an empty dictionary

    fig, axes = plt.subplots(nrows=1, ncols=len(years), figsize=(15, 5))

    for i, year in enumerate(years):
        df_name = region  # Use the specified region
        df = globals()[df_name + '_DR']

        for n in n_values:
            storage_time = 'hours=' + str(n) + '.0'

            DR_sorted = df.loc[df.date.dt.year == year]
            DR_sorted = DR_sorted.sort_values(by=['date'], ascending=True)
            DR_sorted = DR_sorted.sort_values(by=[storage_time], ascending=False)

            DR_sorted['duration'] = 1
            DR_sorted['duration'] = DR_sorted['duration'].cumsum()

            DR_sorted['CumRev'] = DR_sorted[storage_time].cumsum()
            DR_sorted['CumRevNorm'] = DR_sorted['CumRev'] / DR_sorted[storage_time].sum() * 100
            print(year)   
            #print(DR_sorted['CumRevNorm'].iloc[356:366])
            print(DR_sorted['date'].iloc[356:366])

            dict_of_df["df_{}_{}".format(region, n)] = DR_sorted

            # Plot the load duration curve (Revenue vs Percentage of time) on the corresponding subplot
            pltTitle = str(year)
            # Create the line plot
            lineplot = sb.lineplot(x="duration", y="CumRevNorm", data=dict_of_df['df_{}_{}'.format(region, n)], ax=axes[i], label=f'n={n} hours')

        # Set attributes using .set() on the AxesSubplot object
        axes[i].set(title=pltTitle, xlabel='Number of Days', ylabel='Percentage of yearly Revenue (%)')

        axes[i].set_ylim(0, 100)
        axes[i].set_xlim(0, 366)
        axes[i].set_title(pltTitle)
        axes[i].grid(True)  # Turn on gridlines

    plt.suptitle('Normalized Revenue Duration Curves for the NEM')# + region)
    plt.tight_layout()
    plt.legend()
    plt.show()
           



            
def marginalBenefitPlot(regions):
    """
    Plot
    """
    for i, df_name in enumerate(regions):
        name = df_name + '_YtR'
        df = globals()[name]
        normalized_df = df.copy()
        # Calculate the normalization factor as the rightmost column
        #CHANGE:::::: tHIS ASSUMES RH column has max value
        normalization_factor = normalized_df.max(axis=1)

        # Divide each column by the normalization factor to normalize the data
        pcTotal_df = normalized_df.divide(normalization_factor, axis=0) * 100
        #print(pcTotal_df.tail(5).mean().round(2))
        A = pcTotal_df.tail(5).mean().round(2)
        print(A.iloc[:])

        #iloc[:, 0]

        # Plot each year as a different color
        pcTotal_df.tail(10).T.plot.bar(figsize=(10, 6))
        #print(pcTotal_df)
        # Set plot labels and title
        plt.xlabel('Hours of Storage')
        plt.ylabel('Percentage of Maximumue Captured (%)')
        pltTitle = df_name + ' Yearly Revenue Capture Percentage'
        plt.title(pltTitle)
        

        # Display the legend
        plt.legend(title='Year')
        plt.ylim(0,100)
        plt.xlim(0,12)
        plt.grid(True)

        # Show the plot
        plt.show()
        
def marginalBenefitPlot2(regions_dataframes):
    """
    Plot the marginal benefit for each region based on the given DataFrames.
    """
    for df_name, df in regions_dataframes.items():
        normalized_df = df.copy()
        # Calculate the normalization factor as the maximum value across all columns
        normalization_factor = normalized_df.max().max()

        # Divide each column by the normalization factor to normalize the data
        pcTotal_df = normalized_df.divide(normalization_factor) * 100

        # Plot each year as a different color
        pcTotal_df.tail(10).T.plot.bar(figsize=(10, 6))

        # Set plot labels and title
        plt.xlabel('Hours of Storage')
        plt.ylabel('Percentage of Maximum Value Captured')
        pltTitle = df_name + ' Arbitrage Value of Storage as a Percentage of Maximum'
        plt.title(pltTitle)

        # Display the legend
        plt.legend(title='Year')
        plt.ylim(0, 100)
        plt.xlim(0, 12)
        plt.grid(True)

        # Show the plot
        plt.show()
    
    
def calculate_financial_year(row):
    start_year = row['date'].year
    end_year = start_year + 1
    return f'{start_year}-{end_year}'


# Define a function to map months to seasons
def get_season(month):
    if 3 <= month <= 5:
        return 'Autumn'
    elif 6 <= month <= 8:
        return 'Winter'
    elif 9 <= month <= 11:
        return 'Spring'
    else:
        return 'Summer'
    
def seasonalPlots(regions):
    """
    
    """
    for df_name in regions:
        name = df_name + '_DR'
        df = globals()[name]
        df['date'] = pd.to_datetime(df['date'])
        # Extract year and month for creating the season column
        #df['year'] = df.index.year
        #df['month'] = df.index.month
        df['year'] = df['date'].dt.year
        df['month'] = df['date'].dt.month
        
        
        # Create the 'season' column using the get_season function
        df['season'] = df['month'].apply(get_season)

        # Group by year and season, then calculate the sum of revenues
        seasonal_revenues = df.groupby(['year', 'season'])['hours=6.0'].sum().reset_index()

        seasonal_revenues.tail(200).plot(figsize=(10, 6))
        
        
def diurnal_slope(regions, length, hours, a, timeframe, MA):
    """
    
    """
    for df_name in regions:
        name = df_name
        df = globals()[name]
        df_DR = globals()[name + '_DR']
        
        df_DR.set_index('date')
        # Assuming your DataFrame is named df and contains 'date' and 'price' columns
        # Convert the 'date' column to datetime type
        df['SETTLEMENTDATE'] = pd.to_datetime(df['SETTLEMENTDATE'])

        # Extract the date and time components from the 'date' column
        df['day'] = df['SETTLEMENTDATE'].dt.date
        # Group by 'day' and find the minimum and maximum prices for each day
        grouped = df.groupby('day').agg(min_price=('RRP', 'min'), max_price=('RRP', 'max'))

        # Calculate the time difference between minimum and maximum time for each day
        grouped['time_diff'] = (df.groupby('day')['SETTLEMENTDATE'].max() - df.groupby('day')['SETTLEMENTDATE'].min()).dt.total_seconds() / 3600  # Convert seconds to hours

        # Calculate the slope of the price change for each day
        grouped['slope'] = (grouped['max_price'] - grouped['min_price']) / grouped['time_diff']
        hoursStr = 'hours=' + str(hours) + '.0'

        if MA == True:
            grouped['21day MA'] = grouped['slope'].rolling(window=length).mean()
            df_DR['21day MA'] = df_DR[hoursStr].rolling(window=length).mean()
            pltTitle = df_name + ' ' + str(length) + ' Day Moving Average'
        else:
            grouped['21day MA'] = grouped['slope']
            df_DR['21day MA'] = df_DR[hoursStr]
            pltTitle = df_name + ' Daily Revenue and Ramp Rate'
            
        correlation_coefficient = np.corrcoef(grouped['21day MA'], df_DR['21day MA'])[0, 1]
        print(f'Correlation Coefficient (r) = {correlation_coefficient:.2f}')

        current_date = pd.Timestamp.now()
        years_captured = current_date - pd.DateOffset(years=timeframe)
        filtered_df = grouped[grouped.index >= years_captured]
        filtered_df_DR = df_DR[df_DR['date'] >= years_captured]

        #filtered_df['21day MA'].plot()
        #df_DR['21day MA'].plot()
        
        fig, ax1 = plt.subplots(figsize=(12, 6))

        color = 'tab:red'
        ax1.set_xlabel('Year')
        ax1.set_ylabel('$ / KW-hour', color=color)
        lns1 = ax1.plot(filtered_df.index, filtered_df['21day MA'], label='Daily Ramp Up Rate', color=color, alpha=a)
        ax1.tick_params(axis='y', labelcolor=color)
        ax1.grid(False)
        ax1.axis(ymin=0,ymax=60)
        ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

        color = 'tab:blue'
        ax2.set_ylabel('$ / KW', color=color)  # we already handled the x-label with ax1
        lns2 = ax2.plot(filtered_df_DR['date'], filtered_df_DR['21day MA'], label='Value of ' + str(hours) + ' hours of Storage', color=color, alpha=a)
        ax2.tick_params(axis='y', labelcolor=color)
        ax2.grid(False)
        ax2.axis(ymin=0,ymax=5000)

        lns = lns1+lns2
        labs = [l.get_label() for l in lns]
        ax1.legend(lns, labs, loc=0)
        
        fig.tight_layout()  # otherwise the right y-label is slightly clipped
        
        #plt.plot(filtered_df.index, filtered_df['21day MA'], label='Max Min Gradient')
        #plt.plot(filtered_df_DR['date'], filtered_df_DR['21day MA'], label='Value of Storage')
        
        plt.title(pltTitle)
        #plt.xlabel('Year')
        #plt.ylabel('$ / kW')
        plt.show()
        


    return grouped



def valueStoragePlot(regions, storage_time):
    """
    
    """

    # Plot the data
    plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
    storage_string = str(storage_time) + '.0'
    for df_name in regions:
        df = globals()[df_name + '_YtR']
        plt.plot(df.index, df[storage_string], label=df_name)

    # Customize the plot
    plt.title('Value of ' + storage_string + ' hours of storage')
    plt.xlabel('Year')
    plt.ylabel('Revenue ($ / kW-year)')
    plt.legend()
    plt.grid(True)

    # Display the plot
    plt.show()

def totalRevenue(regions, storage_time, years):
    """
    
    """
       # Plot the data
    plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
    for i in storage_time:
        storage_string = str(i) + '.0'
        plt.bar([df_name], df[storage_string].tail(years).sum())

    # Customize the plot
    plt.title('Value of ' + storage_string + ' hours of storage for the last ' + str(years) + ' years')
    plt.xlabel('Location')
    plt.ylabel('Revenue ($ / kW-year)')
    plt.legend()
    plt.grid(True)

    # Display the plot
    plt.show()
    
def totalRevenue2(regions, storage_times, years):
      # Create a figure for the plot
    plt.figure(figsize=(10, 6))

    # Initialize empty lists to store revenue values and x-axis positions
    revenue_values = []
    x_positions = np.arange(len(regions))

    # Loop through the provided regions
    for df_name in regions:
        # Assuming you have data frames like 'df_name_YtR' for each region
        df = globals()[df_name + '_YtR']

        # Initialize an empty list to store revenue values for this region
        region_revenue = []

        for storage_time in storage_times:
            storage_string = str(storage_time) + '.0'

            # Calculate the total revenue for the last 'years' years
            total_revenue = df[storage_string].tail(years).sum()

            # Append the total revenue to the list
            region_revenue.append(total_revenue)

        # Store the revenue values for this region
        revenue_values.append(region_revenue)

    # Plot the data for each region
    bar_width = 0.2  # Adjust the width of the bars as needed
    for i, region in enumerate(regions):
        plt.bar(x_positions + (i * bar_width), revenue_values[i], bar_width, label=region)

    # Customize the plot
    plt.title(f'Total Revenue for {years} Years of Storage')
    plt.xlabel('Location')
    plt.ylabel('Revenue ($ / kW-year)')
    plt.xticks(x_positions + ((len(storage_times) - 1) * bar_width) / 2, regions)
    plt.legend()
    plt.grid(True)

    # Display the plot
    plt.show()


    
    
    
def openNEM(regions):
    """
    """

    dataframes_dict = {}
    
    # Select Region
    for df_name in regions:
        file_path = "OpenNEM/" + df_name + ".csv"
            
        print(f'{df_name} Calling OpenNEM from csv.')
        df = pd.read_csv(file_path)
        df['date'] = pd.to_datetime(df['date'])
        dataframes_dict[df_name] = df
        
        globals()["on_" + df_name] = pd.read_csv(file_path)

    return on_TAS, on_SA, on_QLD, on_NSW, on_VIC, on_AUS


def peakTimesPlot(regions_aus, year_list):
        
    for df_name in regions_aus:
        df = globals()[df_name]
        df['SETTLEMENTDATE'] = pd.to_datetime(df['SETTLEMENTDATE'])
        df['year'] = df['SETTLEMENTDATE'].dt.year
        df['month'] = df['SETTLEMENTDATE'].dt.month
        df['hour'] = df['SETTLEMENTDATE'].dt.hour
        df['minute'] = df['SETTLEMENTDATE'].dt.minute
        df['season'] = df['month'].apply(get_season)

        # Group by year and date, and find highest and lowest prices
        daily_high = df.groupby(['year', df['SETTLEMENTDATE'].dt.date])['RRP'].idxmax()
        daily_low = df.groupby(['year', df['SETTLEMENTDATE'].dt.date])['RRP'].idxmin()
        
        # Count occurrences of each time for highest and lowest prices for each year
        high_time_counts = df.loc[daily_high].groupby(['year', 'hour', 'minute']).size().reset_index(name='count')
        low_time_counts = df.loc[daily_low].groupby(['year', 'hour', 'minute']).size().reset_index(name='count')

        plt.figure(figsize=(12, 6))
        for year in year_list: # df['year'].unique():
            high_year_data = high_time_counts[high_time_counts['year'] == year]
            #print(year)
            #print(high_year_data.describe())
            plt.plot(high_year_data['hour'], high_year_data['count'], label=f'{year}', marker='o')
        plt.xlabel('Time of Day')
        plt.ylabel('Number of Occurrences')
        plt.title('Timing of Daily Highest Prices in ' + df_name)
        plt.xticks(range(24))  # Set x-axis to show all hours
        #plt.xlim(0,23)
        plt.legend()
        plt.show()

        plt.figure(figsize=(12, 6))
        for year in year_list: # df['year'].unique():
            low_year_data = low_time_counts[low_time_counts['year'] == year]
            #print(low_year_data.describe())
            plt.plot(low_year_data['hour'], low_year_data['count'], label=f'{year}', marker='x')
        plt.xlabel('Time of Day')
        plt.ylabel('Number of Occurrences')
        plt.title('Timing of Daily Lowest Prices in ' + df_name)
        plt.xticks(range(24))  # Set x-axis to show all hours
        #plt.xlim(0,23)
        plt.legend()
        plt.show()
        return
    
def peakTimesPlot2(regions_aus, year_list):
    
    for df_name in regions_aus:
        df = globals()[df_name]
        df['SETTLEMENTDATE'] = pd.to_datetime(df['SETTLEMENTDATE'])
        df['year'] = df['SETTLEMENTDATE'].dt.year
        df['month'] = df['SETTLEMENTDATE'].dt.month
        df['hour'] = df['SETTLEMENTDATE'].dt.hour
        df['minute'] = df['SETTLEMENTDATE'].dt.minute
        df['season'] = df['month'].apply(get_season)

        # Group by year and date, and find highest and lowest prices
        daily_high = df.groupby(['year', 'season', df['SETTLEMENTDATE'].dt.date])['RRP'].idxmax()
        daily_low = df.groupby(['year', 'season', df['SETTLEMENTDATE'].dt.date])['RRP'].idxmin()

        # Count occurrences of each time for highest and lowest prices for each year
        high_time_counts = df.loc[daily_high].groupby(['year', 'hour', 'minute', 'season']).size().reset_index(name='count')
        low_time_counts = df.loc[daily_low].groupby(['year', 'hour', 'minute', 'season']).size().reset_index(name='count')
        print(low_time_counts)
        
        plt.figure(figsize=(12, 6))
        for year in year_list: # df['year'].unique():
            high_year_data = high_time_counts[high_time_counts['year'] == year]
            plt.plot(high_year_data['hour'], high_year_data['count'], label=f'{year}', marker='o')
        plt.xlabel('Time of Day')
        plt.ylabel('Number of Occurrences')
        plt.title('Timing of Daily Highest Prices in ' + df_name)
        plt.xticks(range(24))  # Set x-axis to show all hours
        #plt.xlim(0,23)
        plt.legend()
        plt.show()

        plt.figure(figsize=(12, 6))
        for year in year_list: # df['year'].unique():
            low_year_data = low_time_counts[low_time_counts['year'] == year]
            plt.plot(low_year_data['hour'], low_year_data['count'], label=f'{year}', marker='x')
        plt.xlabel('Time of Day')
        plt.ylabel('Number of Occurrences')
        plt.title('Timing of Daily Lowest Prices in ' + df_name)
        plt.xticks(range(24))  # Set x-axis to show all hours
        #plt.xlim(0,23)
        plt.legend()
        plt.show()
        return

def peakTimesPlot3(regions_aus, year_list):
    
    for df_name in regions_aus:
        df = globals()[df_name]
        df['SETTLEMENTDATE'] = pd.to_datetime(df['SETTLEMENTDATE'])
        df['year'] = df['SETTLEMENTDATE'].dt.year
        df['month'] = df['SETTLEMENTDATE'].dt.month
        df['hour'] = df['SETTLEMENTDATE'].dt.hour
        df['minute'] = df['SETTLEMENTDATE'].dt.minute
        df['season'] = df['month'].apply(get_season)

        # Group by year, season, and date, and find highest and lowest prices
        daily_high = df.groupby(['year', 'season', df['SETTLEMENTDATE'].dt.date])['RRP'].idxmax()
        daily_low = df.groupby(['year', 'season', df['SETTLEMENTDATE'].dt.date])['RRP'].idxmin()

        # Count occurrences of each time for highest and lowest prices for each year and season
        high_time_counts = df.loc[daily_high].groupby(['year', 'season', 'hour', 'minute']).size().reset_index(name='count')
        low_time_counts = df.loc[daily_low].groupby(['year', 'season', 'hour', 'minute']).size().reset_index(name='count')
        
        seasons = df['season'].unique()
        plt.figure(figsize=(12, 6))
        for year in year_list:
            for season in seasons:
                high_year_season_data = high_time_counts[(high_time_counts['year'] == year) & (high_time_counts['season'] == season)]
                if not high_year_season_data.empty:
                    label = f'{year} - {season}'
                    plt.plot(high_year_season_data['hour'], high_year_season_data['count'], label=label, marker='o')
        plt.xlabel('Time of Day')
        plt.ylabel('Number of Occurrences')
        plt.title('Timing of Daily Highest Prices in ' + df_name)
        plt.xticks(range(24))  # Set x-axis to show all hours
        plt.legend()
        plt.show()

        plt.figure(figsize=(12, 6))
        for year in year_list:
            for season in seasons:
                low_year_season_data = low_time_counts[(low_time_counts['year'] == year) & (low_time_counts['season'] == season)]
                if not low_year_season_data.empty:
                    label = f'{year} - {season}'
                    plt.plot(low_year_season_data['hour'], low_year_season_data['count'], label=label, marker='x')
        plt.xlabel('Time of Day')
        plt.ylabel('Number of Occurrences')
        plt.title('Timing of Daily Lowest Prices in ' + df_name)
        plt.xticks(range(24))  # Set x-axis to show all hours
        plt.legend()
        plt.show()

        return low_time_counts, high_time_counts
        
def ValuePlotMA(regions, length, hours):
    for df_name in regions:
        # Plot the data
        plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
        df = globals()[df_name + '_DR']
        hoursStr = 'hours=' + str(hours) + '.0'

        df['MA'] = df[hoursStr].rolling(window=length).mean()

        plt.plot(df['date'], df['MA'], label=df_name)

        # Customize the plot
        plt.title(df_name + ' ' + str(length) + ' Day Moving Average Value of ' + str(hours) + ' hours of storage')
        plt.xlabel('Year')
        plt.ylabel('Revenue ($ / kW)')
        plt.legend()
        plt.grid(True, which='both')
        plt.minorticks_on()

        # Display the plot
        plt.show()


In [ ]:
# Highest and Lowest Price graph 
def extraStuff():
    df = VIC
    df['SETTLEMENTDATE'] = pd.to_datetime(df['SETTLEMENTDATE'])
    df['hour'] = df['SETTLEMENTDATE'].dt.hour
    df['minute'] = df['SETTLEMENTDATE'].dt.minute

    # Group by date and find highest and lowest prices
    daily_high = df.groupby(df['SETTLEMENTDATE'].dt.date)['RRP'].idxmax()
    daily_low = df.groupby(df['SETTLEMENTDATE'].dt.date)['RRP'].idxmin()

    # Count occurrences of each time for highest and lowest prices
    high_time_counts = df.loc[daily_high].groupby(['hour', 'minute']).size().reset_index(name='count')
    low_time_counts = df.loc[daily_low].groupby(['hour', 'minute']).size().reset_index(name='count')

    # Plot results
    plt.figure(figsize=(12, 6))
    plt.plot(high_time_counts['hour'], high_time_counts['count'], label='Highest Price', marker='o')
    plt.plot(low_time_counts['hour'], low_time_counts['count'], label='Lowest Price', marker='x')
    plt.xlabel('Hour')
    plt.ylabel('Number of Occurrences')
    plt.title('Occurrences of Highest and Lowest Prices by Hour (Victoria)')
    plt.xticks(range(24))  # Set x-axis to show all hours
    plt.legend()
    plt.grid()
    plt.show()
    
    ###
    #Calculate the number of days from the earliest date

    VIC_DR['days_passed'] = (VIC_DR['date'] - VIC_DR['date'].min()).dt.days

    # Define an interest rate (example: 5% annually)
    interest_rate = 0.05

    # Calculate present value using compound interest formula: PV = FV / (1 + r)^n
    VIC_DR['present_value'] = VIC_DR['hours=6.0'] / (1 - interest_rate) ** (VIC_DR['days_passed'] / 365)

    # Plot the data
    plt.figure(figsize=(10, 6))  # Adjust the figure size as needed

    # Plot DataFrame 2
    plt.plot(VIC_DR['date'], VIC_DR['present_value'], label='VIC Present Value (5%)')
    plt.plot(VIC_DR['date'], VIC_DR['hours=6.0'], label='VIC Revenue')

    # Customize the plot
    plt.title('180 Day Moving Average Diurnal Price Differential')
    plt.xlabel('Year')
    plt.ylabel('Revenue ($ / kW)')
    plt.legend()
    plt.grid(True, which='minor')

    # Display the plot
    plt.show()

    ###
    # Plotting
    plt.figure(figsize=(10, 6))

    on_VIC['date'] = pd.to_datetime(on_VIC['date'])

    # Extract only the month part from the dates
    on_VIC['month'] = on_VIC['date'].dt.month

    # Plot the Diurnal Price Differential from VIC_aMR
    plt.plot(VIC_aMR['month'], VIC_aMR['hours=6.0'], label='Avg. Monthly Revenue')

    # Plot the Renewables Percentage from on_VIC
    sb.lineplot(x="month", y="Renewables — %", data=on_VIC, label='Renewables %')

    fig, ax1 = plt.subplots()

    ax1.plot(VIC_aMR['month'], VIC_aMR['hours=6.0'], label='Avg. Monthly Revenue', color='tab:blue')
    ax1.set_xlabel('Year')
    ax1.set_ylabel('Revenue ($ / kW)', color='tab:blue')
    ax1.tick_params(axis='y', labelcolor='tab:blue')
    ax1.legend(loc='upper left')

    # Create a second y-axis (right side)
    ax2 = ax1.twinx()

    ax2.plot(on_VIC['month'], on_VIC['Renewables — %'], label='%R', color='tab:orange')
    ax2.set_ylabel('% Renewables', color='tab:orange')
    ax2.tick_params(axis='y', labelcolor='tab:orange')
    ax2.legend(loc='upper right')

    # Customize the plot
    plt.title('180 Day Moving Average Diurnal Price Differential and Renewables')
    plt.grid(True)

    # Display the plot
    plt.show()


